#### Getting started With Langchain And Open AI

In this quickstart we'll see how to:

- Set up LangChain, LangSmith, and LangServe
- Utilize the core components of LangChain, including prompt templates, models, and output parsers
- Create a simple application using LangChain
- Monitor your application with LangSmith
- Serve your application using LangServe

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x108ff70b0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x109018dd0> root_client=<openai.OpenAI object at 0x1085151c0> root_async_client=<openai.AsyncOpenAI object at 0x108ff7110> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


In [3]:
result=llm.invoke("What is Agentic AI?")

APIConnectionError: Connection error.

Failed to get info from https://api.smith.langchain.com: LangSmithConnectionError('Connection error caused failure to GET /info in LangSmith API. Please confirm your internet connection. SSLError(MaxRetryError("HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /info (Caused by SSLError(SSLCertVerificationError(1, \'[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)\')))"))\nContent-Length: None\nAPI Key: lsv2_********************************************b1')
Failed to batch ingest runs: langsmith.utils.LangSmithConnectionError: Connection error caused failure to POST https://api.smith.langchain.com/runs/batch in LangSmith API. Please confirm your internet connection. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/batch (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED]

In [ ]:
result

In [6]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert Data Scientist and Gen AI Engineer. Provide me answers based on the asked question "),
        ("user","{query}")

    ]
)
prompt

### Chains
Chains are easily reusable components linked together.

Chains encode a sequence of calls to components like models, document retrievers, other Chains, etc., and provide a simple interface to this sequence.

The Chain interface makes it easy to create apps that are:

Stateful: add Memory to any Chain to give it state,

Observable: pass Callbacks to a Chain to execute additional functionality, like logging, outside the main sequence of component calls,

Composable: combine Chains with other components, including other Chains.

In [ ]:
## chain 
chain=prompt|llm

response=chain.invoke({"query":"Can you tell me something about Genertaive ai vs agentic ai"})
print(response)


In [ ]:
response.content

## Stroutput Parser
The StrOutputParser is a fundamental component in the Langchain framework, designed to streamline the output from language models (LLMs) and ChatModels into a usable string format. This parser is particularly useful when dealing with outputs that may vary in structure, such as strings or messages. It ensures that the output is consistent and easy to handle in subsequent processing steps.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser=StrOutputParser()
chain=prompt|llm|output_parser

response=chain.invoke({"query":"Can you tell me about Langsmith?"})
print(response)

In [15]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
output_parser=JsonOutputParser()
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
output_parser=JsonOutputParser()
chain=prompt|llm|output_parser

response=chain.invoke({"query":"Can you tell me about Langsmith?"})
print(response)

In [ ]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
loader=WebBaseLoader("https://python.langchain.com/docs/introduction/")
loader

In [ ]:
documents=loader.load()
documents

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(documents)
documents


In [25]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [27]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [ ]:
vectorstoredb

In [ ]:
query="Langchain is a framework"
result=vectorstoredb.similarity_search(query)
result[0].page_content